In [36]:
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense,Conv2D,BatchNormalization,MaxPool2D
from keras.layers import Dropout, Flatten
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn import svm
from sklearn.pipeline import Pipeline
# Moduli di scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split, StratifiedShuffleSplit, StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split, StratifiedShuffleSplit
import ml_utilities
import ml_visualization
import ml_utilities
import ml_visualization
import subprocess

In [14]:
TESTS_PATH = "./tests/"
INFO_PATH = "./tests/experiments_info"
DATASET_TEST_PATH = 'DBs/PenDigits/pendigits_te.txt'
DATASET_PATH = 'DBs/PenDigits/pendigits_tr.txt'

In [13]:
def read_data():
    feat_count = 16
    data_path = DATASET_PATH # Impostare il percorso corretto
    patterns, labels = ml_utilities.load_labeled_dataset_from_txt(data_path, feat_count)
    test_path = DATASET_TEST_PATH
    x_test = ml_utilities.load_unlabeled_dataset_from_txt(test_path, feature_count)
    return patterns, labels, x_test

In [12]:
def normalize_data(dataset_patterns, test_x):
    scaler = MinMaxScaler(feature_range = (0,1))
    scaler.fit(dataset_patterns)
    x_train_normalized = scaler.fit_transform(dataset_patterns)
    x_test_normalized = scaler.fit_transform(test_x)

    return x_train_normalized, x_test_normalized

In [11]:
def feature_extractor(x_grid):
    SIZE = x_grid.shape[1]
    activation = 'sigmoid'
    feature = Sequential()

    feature.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 1)))
    feature.add(BatchNormalization())

    feature.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
    feature.add(BatchNormalization())
    feature.add(MaxPool2D())

    feature.add(Flatten())

    return feature

In [50]:

from scipy.stats import loguniform

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [10**x for x in np.arange(-1,3,0.0125)],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf','linear']
        }
    },
    'SVC': {
        'model': SVC(),
        'params' : {
            'C': [10**x for x in np.arange(-1,3,0.0125)],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf','linear'],
            'gamma': [10**x for x in np.arange(-3,1,0.0125)],
            'class_weight':['balanced', None],
            'degree' : [1,2,3,4,5,6]
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params' : {
            'n_neighbors': [1,3,5,7,9,11,13,15,17,19,21,23,25,27],
            'weights': ['uniform','distance'],
            'algorithm': ["ball_tree","kd_tree","brute"],
            'metric': ["minkowski","euclidean","l1", "l2","manhattan"],

        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [10**x for x in np.arange(-1,3,0.0125)],  #Regularization. . Providing only two as LR can be slow
            'penalty': ['l1', 'l2']
        }
    }
}
scores=[]

In [51]:
# Caricamento del dataset
feature_count = 16
dataset_path = 'DBs/PenDigits/pendigits_tr.txt'  # Impostare il percorso corretto

dataset_patterns, dataset_labels = ml_utilities.load_labeled_dataset_from_txt(dataset_path, feature_count)
print('Shape dataset:', dataset_patterns.shape)
print('Shape labels:', dataset_labels.shape)

Shape dataset: (442, 16)
Shape labels: (442,)


In [48]:
!conda create -n xgboost_env -c nvidia -c rapidsai py-xgboost cudatoolkit=10.2
np.random.seed(42)

x_train, y_train, x_test = read_data()
x_grid, x_not_use, y_grid, y_not_use = train_test_split(x_train,y_train, test_size=0.2,random_state=42)
for data_test_size in [0.2]:
    for folds in [1]:
        cross_val = StratifiedShuffleSplit(n_splits=folds, test_size=data_test_size, random_state=42)
        xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', nthread=1,tree_method='gpu_hist')
        params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.02, 0.05]
        }
        for model_name, mp in model_params.items():
            # pipe = Pipeline([('classifier' , mp['model'])])
            grid =  RandomizedSearchCV(xgb,
                                       param_distributions=params,
                                       cv=cross_val,
                                       n_jobs=16,
                                       n_iter=2,
                                       return_train_score=False,
                                       random_state=42)

            grid.fit(x_train,y_train)
            scores.append({
                'data_test_size': data_test_size,
                'folds': folds,
                'model': model_name,
                'best_score': grid.best_score_,
                'mean_test_score': grid.cv_results_['mean_test_score'],
                'best_params': grid.best_params_,
            })

        df = pd.DataFrame(scores,columns=['data_test_size','folds','model','best_score','mean_test_score','best_params',])

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


C:\Users\David\.conda\envs\Test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\xgboost\sklearn.py", line 1250, in fit
    self._Booster = train(
  File "C:\Users\David\.conda\envs\Test\lib\site-packag

XGBoostError: [12:37:58] c:\windows\temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\common\common.h:157: XGBoost version not compiled with GPU support.

In [ ]:
np.random.seed(42)

x_train, y_train, x_test = read_data()
x_grid, x_not_use, y_grid, y_not_use = train_test_split(x_train,y_train, test_size=0.2,random_state=42)
for data_test_size in np.arange(0.1, 1.0, 0.05):
    for folds in  np.arange(1, 10, 1):
        cross_val = StratifiedShuffleSplit(n_splits=folds, test_size=data_test_size, random_state=42)
        for model_name, mp in model_params.items():
            grid =  RandomizedSearchCV(estimator=mp['model'],
                                       param_distributions=mp['params'],
                                       cv=cross_val,
                                       n_jobs=16,
                                       n_iter=2,
                                       return_train_score=False,
                                       random_state=42)

            grid.fit(x_train,y_train)
            scores.append({
                'data_test_size': data_test_size,
                'folds': folds,
                'model': model_name,
                'best_score': grid.best_score_,
                'mean_test_score': grid.cv_results_['mean_test_score'],
                'best_params': grid.best_params_,
            })
        df = pd.DataFrame(scores,columns=['data_test_size','folds','model','best_score','mean_test_score','best_params',])
        print(f"data_test_size= {data_test_size} and folds = {folds}")
        print(f"fit completed")

data_test_size= 0.1 and folds = 1
fit completed


In [ ]:
df